In [ ]:
import json
from fastapi.encoders import jsonable_encoder
import requests
from utils import parse_stream_output

# Function(s) for Calling LLM Endpoint

In [ ]:
def generate_result(
    prompt, api_key_header, stream=False
):
    url = "https://api.obviously.ai/v3/llm/inference"
    headers = {
        "Authorization": api_key_header,
        "Content-Type": "application/json",
    }

    payload = {
        "messages": [
            {
                "role": "user",
                "content": prompt,
            }
        ],
        "additional_params": {"temperature": 0.7, "max_tokens": 256, "stream": stream},
    }
    response = requests.post(url, stream=stream, json=payload, headers=headers)
    if response.status_code == 200:
        
        if stream:
            parse_stream_output(response) # Print out API response as a stream
            return False, 200
            
        else:
            print(response.text) # Print out API response as a JSON string
            return False, 200
    else:
        print(response.text) # Print out API Error response as a JSON string
        return True, response.status_code

# POST Endpoint

In [ ]:
# POST /search/:search_id

req = json.loads(REQUEST)
body = req["body"]  # Request Body Dict
query_params = req.get("args", {}) # Query Param Dict
path_params = req.get("path", {}) # Path Param Dict
headers = req["headers"] # Request Header dict
stream = True if query_params.get("stream") else False

search_id = path_params.get("search_id")

error = False  # Set this to 'True' if you encounter error. Print out any error message post-json-serialization
error_code = 400

api_key_header = headers.get("Authorization")

if not api_key_header:
    error = True
    error_code = 401
    error_response = {"detail": "Unauthorized"}
    print(json.dumps(error_response))  # Print out API Error response as a JSON string

else:
    ########
    ### Call any Pre-processing functions/code below: ###
    ########
    prompt = body.get("prompt")

    # Call obv.ai LLM endpoint with your prompt below:

    error, error_code = generate_result(
        prompt,
        api_key_header,
        stream=stream,
    )
    

    

    
    


# Response codes defined below for endpoint `/search/:search_id`

In [ ]:
# ResponseInfo POST /search/:search_id

if error:
    print(json.dumps({
        "headers": {
            "Content-Type": "application/json"
        },
        "status": error_code
    }))
elif stream:
    print(
        json.dumps(
            {
                "headers": {
                    "Content-Type": "text/event-stream",
                    "x-accel-buffering": "no"
                },
                "status": 200,
            }
        )
    )
else:
    print(json.dumps({
        "headers": {
            "Content-Type": "application/json"
        },
        "status": 200
    }))

# From the `notebooks` directory, run the notebook using command:

```
jupyter kernelgateway --api='kernel_gateway.notebook_http' --seed_uri='main.ipynb' --port 9090
```